![](/Volumes/datamaster/bronze/images/object-model-40d730065eefed283b936a8664f1b247.png)

In [0]:
%python
https://docs.databricks.com/aws/en/database-objects

In [0]:
use catalog datamaster;
create schema if not exists bronze;

-- OR create schema datamaster.bronze;

In [0]:
create table if not exists datamaster.bronze.test (id int, name string, age int);
insert into datamaster.bronze.test values (1, 'Jason', 33);

In [0]:
select * from datamaster.bronze.test;

###Functions

In [0]:
use catalog datamaster;
use schema bronze;

CREATE OR REPLACE TABLE customer_reviews (
  customer_id INT,
  first_name STRING,
  last_name STRING,
  review STRING
);

INSERT INTO customer_reviews VALUES
(1, 'John', 'Doe', 'Amazing product! I love it.'),
(2, 'Jane', 'Smith', 'Not good, I am disappointed.'),
(3, 'Alice', 'Brown', 'Decent quality but can be better.'),
(4, 'Bob', 'Johnson', 'Fantastic service! Highly recommend.'),
(5, 'Charlie', 'Davis', 'Terrible experience, never buying again.');

In [0]:
select * from datamaster.bronze.customer_reviews;

In [0]:
create function function_name(para datatype)
returns datatype
return logic

In [0]:
create function datamaster.bronze.full_name_udf(first_name string, last_name string)
returns string
return concat(first_name, ' ', last_name)

In [0]:
select customer_id, datamaster.bronze.full_name_udf(first_name, last_name) as full_name, review
from datamaster.bronze.customer_reviews

In [0]:
-- Create a complex UDF for sentiment analysis
CREATE FUNCTION datamaster.bronze.sentiment_analysis(review STRING)
RETURNS STRING -- datatype string
RETURN 
    CASE 
        WHEN review LIKE '%amazing%' OR review LIKE '%love%' OR review LIKE '%fantastic%' OR review LIKE '%recommend%' THEN 'Positive'
        WHEN review LIKE '%not good%' OR review LIKE '%disappointed%' OR review LIKE '%terrible%' OR review LIKE '%never buying%' THEN 'Negative'
        ELSE 'Neutral'
    END;

In [0]:
select customer_id, review, datamaster.bronze.sentiment_analysis(review) as sentiment
from datamaster.bronze.customer_reviews

## Views
Virtual Tables
1. Standard View: Persisted (saved)
2. Temp View / Global Temp View

In [0]:
create schema if not exists datamaster.silver;
create schema if not exists datamaster.gold;

In [0]:
-- create view called customer sentiment
-- appears in catalog > datamaster > X > tables; icon has glasses
create or replace view datamaster.silver.customer_sentiment as
select customer_id, review, datamaster.bronze.sentiment_analysis(review) as sentiment
from datamaster.bronze.customer_reviews

In [0]:
select * from datamaster.silver.customer_sentiment

In [0]:
-- gold bc aggregations and grouping happening
create or replace view datamaster.gold.customer_sentiment as
SELECT
  datamaster.bronze.sentiment_analysis(review) AS sentiment,
  COUNT(*) AS count
FROM datamaster.bronze.customer_reviews
GROUP BY sentiment_analysis(review)

In [0]:
select * from datamaster.gold.customer_sentiment;

-- output table:
-- sentiment | count
-- neutral | 1
-- negative | 2
-- positive | 2